In [ ]:
# import sys

# class Logger(object):
#     def __init__(self, filename="tensorflow_logs.log"):
#         self.terminal = sys.stdout
#         self.log = open(filename, "a")

#     def write(self, message):
#         self.terminal.write(message)
#         self.log.write(message)

#     def flush(self):
#         # 이 메소드는 flush() 인터페이스를 유지하기 위한 것입니다.
#         pass

# sys.stdout = Logger("tensorflow_logs.log")

# # 테스트 출력
# print("This will be written to both console and file")


In [ ]:
import numpy as np

datasets = np.load('npz/ObjectDetection.npz', allow_pickle=True)
images, numbers, bboxes = datasets['images'], datasets['numbers'], datasets['bboxes']

max_label_length = 4
labels = []
for num in numbers:
    cls = [1] * num if num != 0 else [0]
    cls += [0] * (max_label_length - len(cls))
    labels.append(cls)

# labels = np.array(labels)
expected_image_shape = images[9000].shape
expected_bbox_shape = bboxes[9000].shape
expected_label_length = bboxes[9000].shape

non_zero_indices = np.where(numbers > 2)[0]

images_filtered = images[non_zero_indices]
bboxes_filtered = bboxes[non_zero_indices]
labels_filtered = np.array(labels)[non_zero_indices]
numbers_filtered = numbers[non_zero_indices]
# labels = np.array(labels)


for i in range(len(images_filtered)):
    image = images_filtered[i]
    bbox = bboxes_filtered[i]
    label = labels_filtered[i]

    # 이미지 형태와 데이터 타입 검증
    if image.shape != expected_image_shape or image.dtype != np.uint8:
        print(f"이미지 {i}의 형태 또는 데이터 타입이 잘못되었습니다: 형태={image.shape}, 데이터 타입={image.dtype}")

    # 바운딩 박스 형태와 데이터 타입 검증
    if bbox.shape != expected_bbox_shape or bbox.dtype != np.float64:
        print(f"바운딩 박스 {i}의 형태 또는 데이터 타입이 잘못되었습니다: 형태={bbox.shape}, 데이터 타입={bbox.dtype}")

    # # 레이블 길이와 데이터 타입 검증
    # if len(label) != expected_label_length or not all(isinstance(x, int) for x in label):
    #     print(f"레이블 {i}의 길이 또는 데이터 타입이 잘못되었습니다: 길이={len(label)}, 레이블={label}")

# print(images.shape, numbers.shape, bboxes.shape, len(labels))

# print(images.max(), images.min())
# print(bboxes[9000:9010])
# print(labels[9000:9010])


dataset = {
    'images' : images_filtered,
    'bboxes' : bboxes_filtered,
    'cls' : labels_filtered
}

print(dataset['images'].shape)
print(dataset['bboxes'].shape)
print(dataset['cls'])
non_zero_indices = np.where(numbers_filtered == 0)[0]
print("non_zero_indices: ", non_zero_indices)


In [ ]:
# print(labels[9000])

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
images = dataset['images']
# numbers =dataset['numbers']
bboxes = dataset['bboxes']
cls = dataset['cls']

boxes = bboxes[0]
plt.figure(figsize = (8, 8))
plt.axis('off')
image = images[0]
print(image.shape)
print(image.shape[0])
print(image.shape[0])
plt.imshow(images[0])
ax = plt.gca()
boxes = tf.stack([
	boxes[:, 0] * images.shape[2],
	boxes[:, 1] * images.shape[1],
	boxes[:, 2] * images.shape[2],
	boxes[:, 3] * images.shape[1]], axis = -1
)

print(boxes)

for box in boxes:
	xmin, ymin = box[:2]
	w, h = box[2:] - box[:2]
	patch = plt.Rectangle(
		[xmin, ymin], w, h, fill = False, edgecolor = [1, 0, 0], linewidth = 2
	)
	ax.add_patch(patch)
plt.show()
print(cls[0])

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf

IMG_SIZE_WIDTH = images.shape[2]
IMG_SIZE_HEIGHT = images.shape[1]
N_DATA = images.shape[0]
N_TRAIN = int(images.shape[0] * 0.7)
N_VAL = images.shape[0] - N_TRAIN
LOG_DIR = 'ObjectDetectionLog'

cur_dir = os.getcwd()
tfr_dir = os.path.join(cur_dir, 'tfrecord/ObjectDetection')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'od_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'od_val.tfr')

print("IMG_SIZE_WIDTH:  ", IMG_SIZE_WIDTH)
print("IMG_SIZE_HEIGHT: ", IMG_SIZE_HEIGHT)
print("N_DATA:          ", N_DATA)
print("N_TRAIN:         ", N_TRAIN)



shuffle_list = list(range(N_DATA))
random.shuffle(shuffle_list)

train_idx_list = shuffle_list[:N_TRAIN]
val_idx_list = shuffle_list[N_TRAIN:]


cur_dir = os.getcwd()
tfr_dir = os.path.join(cur_dir, 'tfrecord/ObjectDetection')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'od_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'od_val.tfr')

writer_train = tf.io.TFRecordWriter(tfr_train_dir)
writer_val = tf.io.TFRecordWriter(tfr_val_dir)

In [ ]:
# images.max(), images.min()

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [value]))

def _float_feature(value):
    return tf.train.Feature(float_list = tf.train.FloatList(value = value))

def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))


In [ ]:

for idx in train_idx_list:
    bbox = bboxes[idx]
    xmin, ymin, xmax, ymax = bbox[:, 0], bbox[:, 1], bbox[:, 2], bbox[:, 3]
    bbox = np.stack([xmin, ymin, xmax, ymax], axis=-1).flatten()

    image = images[idx]
    bimage = image.tobytes()

    number = numbers[idx]
    class_id = cls[idx]
    # print(len(cls))
    serialized_cls = tf.io.serialize_tensor(tf.constant(class_id, dtype=tf.int32)).numpy()

    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(bimage),
        'bbox': _float_feature(bbox),
        'label': _bytes_feature(serialized_cls),
        # 'number': _int64_feature(number)
    }))
    
    writer_train.write(example.SerializeToString())
writer_train.close()


In [ ]:
import os
import random
import tensorflow as tf

AUTOTUNE = tf.data.AUTOTUNE

RES_HEIGHT =24
RES_WIDTH = 32
# N_EPOCHS = 100
# N_BATCH = 8
# LR = 0.0005


def _parse_function(tfrecord_serialized):
    features = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'bbox': tf.io.VarLenFeature(tf.float32),  
        'label': tf.io.FixedLenFeature([], tf.string),
        # 'number': tf.io.FixedLenFeature([], tf.int64)
    }

    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)

    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [RES_HEIGHT, RES_WIDTH, 1])
    image = image / tf.reduce_max(image)
    image = tf.cast(image, tf.float32) 
    # image = image / tf.reduce_max(image)

    bbox = tf.sparse.to_dense(parsed_features['bbox']) 
    bbox = tf.cast(bbox, tf.float32)
    # num_boxes = tf.shape(bbox)[0] // 4
    bbox = tf.reshape(bbox, [-1, 4])

    serialized_cls = parsed_features['label']
    label = tf.io.parse_tensor(serialized_cls, out_type=tf.int32)
    
    # number = tf.cast(parsed_features['number'], tf.int64)
    return image, bbox, label

# cur_dir = os.getcwd()
# tfr_dir = os.path.join(cur_dir, 'tfrecord/ObjectDetection')
# tfr_train_dir = os.path.join(tfr_dir, 'od_train.tfr')
train_dataset = tf.data.TFRecordDataset(tfr_train_dir)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=AUTOTUNE)
# train_dataset = train_dataset.shuffle(buffer_size=N_TRAIN).prefetch(AUTOTUNE).batch(N_BATCH, drop_remainder=True)

val_dataset = tf.data.TFRecordDataset(tfr_val_dir)
val_dataset = val_dataset.map(_parse_function, num_parallel_calls=AUTOTUNE)


In [ ]:
import matplotlib.pyplot as plt

for image, bbox, label in train_dataset.take(1):
    image = image.numpy()
    print(image.shape)
    # print(label)
    # # plt.axis('off')
    # plt.imshow(image)
    # ax = plt.gca()  
    # print(bbox)

    # boxes = tf.stack(
    # 	[
    # 	 bbox[:,0] * image.shape[1],
    # 	 bbox[:,1] * image.shape[0],
    # 	 bbox[:,2] * image.shape[1],
    # 	 bbox[:,3] * image.shape[0]
    # 	], axis = -1
    # )
    # print(image.shape)
    # print(boxes)
    # for box in boxes:
    #     xmin, ymin = box[:2]
    #     w, h = box[2:] - box[:2]
    #     patch = plt.Rectangle(
    #         [xmin, ymin], w, h, fill=False, edgecolor=[1, 0, 0], linewidth=2
    #     )
    #     ax.add_patch(patch)
    # plt.show()

In [ ]:
def convert_to_xywh(boxes):    
    return tf.concat(
        [(boxes[..., :2] + boxes[..., 2:]) / 2.0, boxes[..., 2:] - boxes[..., :2]],
        axis=-1
    )

In [ ]:
def convert_to_corners(boxes):
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2.0, boxes[..., :2] + boxes[..., 2:] / 2.0],
        axis=-1
    )



In [ ]:
for image, bbox, label in train_dataset.take(1):
    print(bbox)
    print(convert_to_xywh(bbox))

In [ ]:
def resize_and_pad_image(image, min_side=96, max_side=128):
    
    image_shape = tf.cast(tf.shape(image)[:2], dtype = tf.float32)
    # print(f"image_shape: {image_shape}")
    ratio = min_side / tf.reduce_min(image_shape)
    # print(f"ratio: {ratio}")
    if ratio * tf.reduce_max(image_shape) > max_side:
      ratio = max_side / tf.reduce_max(image_shape)
    # print(f"ratio: {ratio}")

    new_image_shape = ratio * image_shape
    # print(f"new_image_shape: {new_image_shape}")

    image = tf.image.resize(image, 
                            tf.cast(new_image_shape, dtype=tf.int32), 
                            method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # print(f"image: {image.shape}")

    return image, new_image_shape, ratio


In [ ]:
def preprocess_data(image, gt_boxes, cls_ids):
    # image = sample["images"]
    # bbox = sample["bboxes"]
    cls_ids = tf.cast(cls_ids, dtype = tf.int32)
    image, image_shape, _ = resize_and_pad_image(image)
    
    bbox = tf.stack([
        gt_boxes[:, 0] * image_shape[1],
        gt_boxes[:, 1] * image_shape[0],
        gt_boxes[:, 2] * image_shape[1],
        gt_boxes[:, 3] * image_shape[0]],
        axis = -1
    )
    
    bbox = convert_to_xywh(bbox)
    print(image.shape)
    print(bbox.shape)
    print(cls_ids.shape)
    return image, bbox, cls_ids

In [ ]:
for image, bbox, label in train_dataset.take(1):
    resize_and_pad_image(image)

In [ ]:
for image, bbox, label in train_dataset.take(1):
    print(label)
    img, bbox, class_id = preprocess_data(image, bbox, label)
    print(img.shape) # (72, 96, 1)

    anchor_img = np.zeros((*img.shape[:2], 3), dtype=np.uint8)
    print(anchor_img.shape)

    print(bbox)

    strides = [4, 8, 16, 32]
    colors = {
        4: [0, 255, 0],  # 초록색
        8: [0, 0, 255],  # 파란색
        16: [255, 0, 0],   # 빨간색
        32:[255, 255, 255],  # 노란색
        # 64:[255, 255, 0],  # 노란색
    }

    for stride in strides:
        color = colors[stride]
        for y in range(0, anchor_img.shape[0], stride):
            for x in range(0, anchor_img.shape[1], stride):
                anchor_img[y, x, :] = color

    # 이미지 표시
    plt.imshow(img, alpha=1)  
    plt.imshow(anchor_img, alpha=0.5) 
    plt.axis('off')
    plt.show()
    print(tf.reduce_max(image), tf.reduce_min(image))

In [ ]:
for image, bbox, label in train_dataset.take(1):
    img, box, label = preprocess_data(image, bbox, label)
    print(label)
    print(box)
    print(tf.reduce_max(image), tf.reduce_min(image))
    # 이미지 시각화
    plt.imshow(img)
    ax = plt.gca()
    width = img.shape[1]
    height = img.shape[0]
    print("width: ", width)
    print("height: ", height)
    boxes = tf.stack(
        [
            (box[:, 0] - 0.5 * box[:, 2])  ,  # xmin = x_center - width/2
            (box[:, 1] - 0.5 * box[:, 3])  ,  # ymin = y_center - height/2
            (box[:, 0] + 0.5 * box[:, 2])  ,  # xmax = x_center + width/2
            (box[:, 1] + 0.5 * box[:, 3])     # ymax = y_center + height/2
        ], axis=-1
    )
    print("bbox: ", boxes)
    # 각 바운딩 박스에 대해 반복하여 그리기
    for box in boxes[2:3]:
        xmin, ymin, xmax, ymax = box
        # print("xmin, ymin: ", xmin, ymin)
        print(box)
        w, h = xmax - xmin, ymax - ymin
        patch = plt.Rectangle(
            [xmin, ymin], w, h, fill=False, edgecolor=[1, 0, 0], linewidth=2
        )
        ax.add_patch(patch)

    plt.show()
    print(label)


In [ ]:
# ((7, 24192, 1) vs (None, 24624, 1)).

class AnchorBox:
    def __init__(self):
        self.aspect_ratios = [0.5, 0.75, 1.0, 1.25]        
        self.scales = [2** x for x in [1/3, 2/3]]
        self._num_anchors = len(self.aspect_ratios) * len(self.scales)
        self._strides = [2 ** i for i in range(2, 8)]
        self._areas = [x ** 2 for x in [8.0, 16.0, 32.0, 64.0, 128.0, 256.0]]
        self._anchor_dims = self._compute_dims()

    def _compute_dims(self):
        anchor_dims_all = []

        for area in self._areas:
            anchor_dims = []
            for ratio in self.aspect_ratios: 
                anchor_height = tf.math.sqrt(area / ratio)
                anchor_width = area / anchor_height
                dims = tf.reshape(
                    tf.stack([anchor_width, anchor_height], axis = -1), [1, 1, 2]
                )
                for scale in self.scales: 
                    anchor_dims.append(scale * dims) 
            anchor_dims_all.append(tf.stack(anchor_dims, axis = -2))
        return anchor_dims_all 
    
    def _get_anchors(self, feature_height, feature_width, level):
        rx = tf.range(feature_width, dtype = tf.float32) + 0.5
        ry = tf.range(feature_height, dtype = tf.float32) + 0.5

        centers = tf.stack(tf.meshgrid(rx, ry), axis = -1) * self._strides[level - 2]
        # print(f"centers: {centers}")
        centers = tf.expand_dims(centers, axis = -2)
        # print(f"centers: {centers}")
        centers = tf.tile(centers, [1, 1, self._num_anchors, 1])
        # print(f"centers: {centers}")

        dims = tf.tile(
            self._anchor_dims[level - 2], [feature_height, feature_width, 1, 1] 
        )
        # print(f"dims: {dims}")

        
        anchors = tf.concat([centers, dims], axis=-1) 
        # print(f"anchors: {anchors}")

        # print(f"{tf.reshape(anchors, [feature_height * feature_width * self._num_anchors, 4]).shape}")

        return tf.reshape(
            anchors, [feature_height * feature_width * self._num_anchors, 4]
        )

    def get_anchors(self, image_height, image_width):
        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / 2 ** i), # 올림
                tf.math.ceil(image_width / 2 ** i),
                i
            )
            for i in range(2, 8)
        ]

        anchors = tf.concat(anchors, axis=0)

        # 앵커 박스의 좌표를 이미지 크기 내로 제한
        anchors = tf.clip_by_value(anchors, 0, [image_height, image_width, image_height, image_width])
        return tf.concat(anchors, axis=0)
        # return anchors

In [ ]:
anchors = AnchorBox()
anchor = anchors.get_anchors(96, 128)

has_negative_values = tf.reduce_any(tf.less(anchor, 0))
print("Anchor 음수 값:", has_negative_values.numpy())

# print(anchor)
print(anchor.shape)
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_bounding_boxes(data, num_samples):
    fig, ax = plt.subplots()

    plt.imshow(img)
    # print(img.shape)
    data_np = data.numpy()

    if len(data) > num_samples:
        sampled_indices = np.random.choice(len(data), num_samples, replace=False)
        sample_data = data_np[sampled_indices]
    else : 
        sample_data = data_np
    print(sample_data)
    for center_x, center_y, width, height in sample_data:
        top_left_x = center_x - width / 2
        top_left_y = center_y - height / 2

        rect = patches.Rectangle((top_left_x, top_left_y), width, height, linewidth=0.8, edgecolor='white', facecolor='none')
        ax.add_patch(rect)
    
    plt.show()

draw_bounding_boxes(anchor, 1)


# 24192

In [ ]:
def compute_iou(boxes1, boxes2):
    boxes1_corners = convert_to_corners(boxes1)
    print("boxes1: ", boxes1)
    print("boxes1_corners: ", boxes1_corners)
    boxes2_corners = convert_to_corners(boxes2)
    print("boxes2: ", boxes2)
    print("boxes2_corners: ", boxes2_corners)
    lu = tf.maximum(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
    print("lu: ", lu)
    rd = tf.minimum(boxes1_corners[:, None, 2:], boxes2_corners[:, 2:])  
    print("rd: ", rd)

    

    intersection = tf.maximum(rd - lu, 0.0)
    print("intersection: ", intersection)
    intersection_area = intersection[:, :, 0] * intersection[:, :, 1]
    print("intersection_area: ", intersection_area)
    boxes1_area = boxes1[:, 2] * boxes1[:, 3]
    boxes2_area = boxes2[:, 2] * boxes2[:, 3]
    union_area = tf.maximum(boxes1_area[:, None] + boxes2_area - intersection_area, 1e-8)
    print("union_area: ", union_area)
    return tf.clip_by_value(intersection_area / union_area, 0.0, 1.0)


In [75]:
b1 = np.array([[70, 62, 16, 16]]) 
b1c = convert_to_corners(b1)
print(b1c)

b2 = np.array([[76, 72, 32, 40]])
b2c = convert_to_corners(b2)
print(b2c)


tf.Tensor([[62. 54. 78. 70.]], shape=(1, 4), dtype=float64)
tf.Tensor([[60. 52. 92. 92.]], shape=(1, 4), dtype=float64)


In [76]:
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# # 주어진 바운딩 박스 데이터
# box1 = [62, 54, 78, 70]  # [x_min, y_min, x_max, y_max]
# box2 = [60, 52, 82, 82]

# # 그림 생성
# fig, ax = plt.subplots()

# # 첫 번째 바운딩 박스 추가
# rect1 = patches.Rectangle((box1[0], box1[1]), box1[2] - box1[0], box1[3] - box1[1], 
#                           linewidth=2, edgecolor='blue', facecolor='none')
# ax.add_patch(rect1)

# # 두 번째 바운딩 박스 추가
# rect2 = patches.Rectangle((box2[0], box2[1]), box2[2] - box2[0], box2[3] - box2[1], 
#                           linewidth=2, edgecolor='red', facecolor='none')
# ax.add_patch(rect2)

# # 축 범위 설정
# ax.set_xlim(0, 90)
# ax.set_ylim(0, 90)

# # 그림 표시
# plt.show()



In [77]:
# GA = np.array([[70, 62, 16, 16]])
# GT = np.array([[76, 72, 32, 40]])

# print(compute_iou(GA, GT))

In [78]:
class LabelEncoder:
    def __init__(self):
        self._anchor_box = AnchorBox()
        self._box_variance = tf.convert_to_tensor(
            [0.1, 0.1, 0.2, 0.2], dtype=tf.float32)
    
    def _match_anchor_boxes(self, anchor_boxes, gt_boxes, match_iou = 0.4, ignore_iou = 0.3):
        iou_matrix = compute_iou(anchor_boxes, gt_boxes)
        # print("iou_matrix:  ", iou_matrix)
        max_iou = tf.reduce_max(iou_matrix, axis=1)
        # print("max_iou:  ", max_iou)


        matched_gt_idx = tf.argmax(iou_matrix, axis = 1)
        # print("matched_gt_idx:  ", matched_gt_idx)

    
        positive_mask = tf.greater_equal(max_iou, match_iou)
        # print("positive_mask:  ", positive_mask)
        negative_mask = tf.less(max_iou, ignore_iou)
        # print("negative_mask:  ", negative_mask)

        ignore_mask = tf.logical_not(tf.logical_or(positive_mask, negative_mask))
        # print("ignore_mask:  ", ignore_mask)
        return (
            matched_gt_idx,
            tf.cast(positive_mask, dtype = tf.float32),
            tf.cast(ignore_mask, dtype = tf.float32),
        )
    
    def _compute_box_target(self, anchor_boxes, matched_gt_boxes):
        box_target = tf.concat(
            [
                (matched_gt_boxes[:, :2] - anchor_boxes[:, :2]) / anchor_boxes[:, 2:],
                tf.math.log(matched_gt_boxes[:, 2:] / anchor_boxes[:, 2:])
            ],
            axis = -1,
        )
        # print("box_target:  ", box_target)
        box_target = box_target / self._box_variance
        # print("box_target:  ", box_target)
        return box_target
    


    def _encode_sample(self, image_shape, gt_boxes, cls_ids):        
        anchor_boxes = self._anchor_box.get_anchors(image_shape[1], image_shape[2])
        # print("anchor_boxes  : ", anchor_boxes)
        cls_ids = tf.cast(cls_ids, dtype=tf.float32)
        # print("cls_ids", cls_ids)
        matched_gt_idx, positive_mask, ignore_mask = self._match_anchor_boxes(
            anchor_boxes, gt_boxes
        )
        # print("matched_gt_idx:  ", matched_gt_idx)
        # print("positive_mask:  ", positive_mask)
        # print("ignore_mask:  ", ignore_mask)

        matched_gt_boxes = tf.gather(gt_boxes, matched_gt_idx)

        # print("matched_gt_boxes:  ", matched_gt_boxes)
        
        
        
        box_target = self._compute_box_target(anchor_boxes, matched_gt_boxes)
        # print("box_target:  ", box_target)



        matched_gt_cls_ids = tf.gather(cls_ids, matched_gt_idx)
        # print("matched_gt_cls_ids:  ", matched_gt_cls_ids)
        
        cls_target = tf.where(
            tf.not_equal(positive_mask, 1.0), -1.0, matched_gt_cls_ids
        )
        # print("cls_target:  ", cls_target)

        cls_target = tf.where(tf.equal(ignore_mask, 1.0), -2.0, cls_target)
        # print("cls_target:  ", cls_target)

        cls_target = tf.expand_dims(cls_target, axis=-1)
        # print("cls_target:  ", cls_target)


        label = tf.concat([box_target, cls_target], axis=-1)
        # print("label:  ", label)
        return label

    def encode_batch(self, batch_images, gt_boxes, cls_ids):       
        images_shape = tf.shape(batch_images)
        # print("images_shape:  ", images_shape)
        batch_size = images_shape[0]
        # print("batch_size:  ", batch_size)

        labels = tf.TensorArray(dtype=tf.float32, size=batch_size, dynamic_size=True)
        # print("labels:  ", labels)
        # batch_size_val = batch_size.numpy()
        for i in range(1):
            label = self._encode_sample(images_shape, gt_boxes[i], cls_ids[i])
            # print("label:  ", label)
            labels = labels.write(i, label)
        return batch_images, labels.stack()

In [79]:
print("Eager execution: ", tf.executing_eagerly())
if not tf.executing_eagerly():
    tf.compat.v1.enable_eager_execution()

Eager execution:  True


In [80]:
# label_encoder = LabelEncoder()

# for image, bbox, label in train_dataset.take(1):
#     img, box, label = preprocess_data(image, bbox, label)
#     img = np.expand_dims(img, axis = 0)
#     box = np.expand_dims(box, axis = 0)
#     label = np.expand_dims(label, axis = 0)

#     print(img.shape, box.shape, label.shape)
#     label_encoder.encode_batch(img, box, label)
    

In [81]:
def add_axis(img, box, label):
    img = np.expand_dims(img, axis = 0)
    box = np.expand_dims(box, axis = 0)
    label = np.expand_dims(label, axis = 0)
    return img, box, label

In [82]:
label_encoder = LabelEncoder()

autotune = tf.data.AUTOTUNE
num_classes = 1
batch_size = 7

train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=autotune)
# train_dataset = train_dataset.shuffle(batch_size * 8)

train_dataset = train_dataset.padded_batch(
    batch_size=batch_size, 
    padded_shapes = ([96, 128, 1], [4, 4], [4]),
    padding_values=(0.0, 1e-8, -2), 
    drop_remainder=True
)


# val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=autotune)
# val_dataset = val_dataset.padded_batch(
#     batch_size=batch_size, 
#     padded_shapes = ([96, 128, 1], [4, 4], [4]),
#     padding_values=(0.0, 1e-8, 0), 
#     drop_remainder=True
# )
# val_dataset = val_dataset.map(
#     label_encoder.encode_batch, num_parallel_calls=autotune
# )

# val_dataset = val_dataset.apply(tf.data.experimental.ignore_errors())
# val_dataset = val_dataset.prefetch(autotune)

(None, None, 1)
(None, 4)
<unknown>


In [83]:
train_dataset = train_dataset.map(
    label_encoder.encode_batch, num_parallel_calls=autotune
)
# train_dataset = train_dataset.apply(tf.data.experimental.ignore_errors())
train_dataset = train_dataset.prefetch(autotune)

boxes1:  Tensor("concat_7/concat:0", shape=(8200, 4), dtype=float32)
boxes1_corners:  Tensor("concat_8:0", shape=(8200, 4), dtype=float32)
boxes2:  Tensor("strided_slice_1:0", shape=(4, 4), dtype=float32)
boxes2_corners:  Tensor("concat_9:0", shape=(4, 4), dtype=float32)
lu:  Tensor("Maximum:0", shape=(8200, 4, 2), dtype=float32)
rd:  Tensor("Minimum:0", shape=(8200, 4, 2), dtype=float32)
intersection:  Tensor("Maximum_1:0", shape=(8200, 4, 2), dtype=float32)
intersection_area:  Tensor("mul_18:0", shape=(8200, 4), dtype=float32)
union_area:  Tensor("Maximum_2:0", shape=(8200, 4), dtype=float32)


In [84]:
# train_dataset에서 하나의 배치를 가져옵니다
positive_count = []
negative_count = []
ignore_count = []
for batch in train_dataset.take(5):
    # 배치에서 이미지와 레이블을 추출합니다
    images, labels = batch
    print(labels.shape)

    # labels 텐서에서 positive, negative, ignore 값의 개수를 계산
    positive_count = tf.reduce_sum(tf.cast(tf.equal(labels[:, :, 4], 1.0), tf.int32))
    negative_count = tf.reduce_sum(tf.cast(tf.equal(labels[:, :, 4], -1.0), tf.int32))
    ignore_count = tf.reduce_sum(tf.cast(tf.equal(labels[:, :, 4], -2.0), tf.int32))

    print("Positive 개수:", positive_count.numpy())
    print("Negative 개수:", negative_count.numpy())
    print("Ignore 개수:", ignore_count.numpy())


(7, 8200, 5)
Positive 개수: 52
Negative 개수: 8048
Ignore 개수: 100
(7, 8200, 5)
Positive 개수: 137
Negative 개수: 7866
Ignore 개수: 197
(7, 8200, 5)
Positive 개수: 80
Negative 개수: 8023
Ignore 개수: 97
(7, 8200, 5)
Positive 개수: 66
Negative 개수: 8003
Ignore 개수: 131
(7, 8200, 5)
Positive 개수: 94
Negative 개수: 7997
Ignore 개수: 109


In [85]:
# for batch in train_dataset.take(1):
#     images, labels = batch
#     print("원본 레이블 shape:", labels.shape)

#     # ignore 상태가 아닌 앵커 박스만 필터링
#     mask = tf.not_equal(labels[:, :, 4], -2.0)
#     filtered_labels = tf.boolean_mask(labels, mask)

#     # 필터링된 레이블을 원하는 크기로 조절
#     # 예: 36288개로 조절
#     desired_count = 36288
#     filtered_labels = filtered_labels[:desired_count, :]
#     print("조정된 레이블 shape:", filtered_labels.shape)


In [86]:
# # def cut_labels(images, labels):
# #     # ignore 상태가 아닌 앵커 박스만 필터링
# #     mask = tf.not_equal(labels[:, 4], -2.0)
# #     filtered_labels = tf.boolean_mask(labels, mask)

# #     # 필터링된 레이블을 원하는 크기로 조절
# #     desired_count = 36288
# #     # 주의: 여기서 filtered_labels의 크기가 desired_count보다 작을 수 있으므로, 적절한 처리가 필요합니다.
# #     if tf.shape(filtered_labels)[0] > desired_count:
# #         filtered_labels = filtered_labels[:desired_count, :]

# #     return images, filtered_labels


def cut_labels(images, labels):
    # ignore 상태가 아닌 앵커 박스만 필터링
    mask = tf.not_equal(labels[:, 4], -2.0)
    filtered_labels = tf.boolean_mask(labels, mask)

    # 필터링된 레이블을 원하는 크기로 조절
    desired_count = 8064
    current_count = tf.shape(filtered_labels)[0]

    # 필요한 경우 레이블을 패딩
    if current_count < desired_count:
        # 누락된 개수만큼 ignore 상태(-2.0)로 패딩
        padding_count = desired_count - current_count
        padding = tf.fill([padding_count, 5], -2.0)  # -2.0으로 채워진 텐서 생성
        filtered_labels = tf.concat([filtered_labels, padding], axis=0)
    else:
        filtered_labels = filtered_labels[:desired_count, :]

    return images, filtered_labels




# # # 원본 train_dataset에 process_labels 함수 적용
train_dataset = train_dataset.unbatch().map(cut_labels).batch(batch_size)
# # # val_dataset = val_dataset.unbatch().map(cut_labels).batch(batch_size)

# # # buffer_size = 1000
# # # 필요한 경우 배치 크기, 셔플, 반복 등을 적용
# # # train_dataset = new_train_dataset.batch(batch_size).shuffle(buffer_size).repeat()
# # # val_dataset = new_val_dataset.batch(batch_size).shuffle(buffer_size).repeat()


In [87]:
# train_dataset에서 하나의 배치를 가져옵니다
positive_count = []
negative_count = []
ignore_count = []
for batch in train_dataset.take(4):
    # 배치에서 이미지와 레이블을 추출합니다
    images, labels = batch
    labels[:, :, 4:]
    print(tf.reduce_max(images), tf.reduce_min(images))
    print(tf.reduce_max(labels), tf.reduce_min(labels))

    # labels 텐서에서 positive, negative, ignore 값의 개수를 계산
    positive_count = tf.reduce_sum(tf.cast(tf.equal(labels[:, :, 4], 1.0), tf.int32))
    
    # pad_count = tf.reduce_sum(tf.cast(tf.equal(labels[:, :, 4], 2.0), tf.int32))

    negative_count = tf.reduce_sum(tf.cast(tf.equal(labels[:, :, 4], -1.0), tf.int32))
    ignore_count = tf.reduce_sum(tf.cast(tf.equal(labels[:, :, 4], -2.0), tf.int32))

    print("Positive 개수:", positive_count.numpy())
    print("Negative 개수:", negative_count.numpy())
    print("Ignore 개수:", ignore_count.numpy())
    # print("Pad 개수:", pad_count.numpy())


tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(106.63389, shape=(), dtype=float32) tf.Tensor(-68.77227, shape=(), dtype=float32)
Positive 개수: 52
Negative 개수: 8012
Ignore 개수: 0
tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(33.67386, shape=(), dtype=float32) tf.Tensor(-103.82774, shape=(), dtype=float32)
Positive 개수: 137
Negative 개수: 7866
Ignore 개수: 61
tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(129.08313, shape=(), dtype=float32) tf.Tensor(-84.30149, shape=(), dtype=float32)
Positive 개수: 80
Negative 개수: 7984
Ignore 개수: 0
tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(115.05237, shape=(), dtype=float32) tf.Tensor(-84.30149, shape=(), dtype=float32)
Positive 개수: 66
Negative 개수: 7998
Ignore 개수: 0


In [88]:
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# # train_dataset에서 하나의 배치를 가져옵니다
# for batch in train_dataset.take(1):
#     images, labels = batch
    
#     # 첫 번째 이미지를 선택
#     image = images[0]
#     image_height, image_width, _ = image.shape

#     # 첫 번째 이미지에 대한 레이블
#     image_labels = labels[0]

#     # 첫 번째 Positive 바운딩 박스 추출
#     positive_boxes = image_labels[tf.equal(image_labels[:, 4], 1.0)]
#     if tf.shape(positive_boxes)[0] > 0:  # Positive 바운딩 박스가 있는 경우에만
#         first_box = positive_boxes[15]  # 첫 번째 바운딩 박스
        
#         boxes = tf.stack(
#         [
#             (first_box[0] - 0.5 * first_box[2])  ,  # xmin = x_center - width/2
#             (first_box[1] - 0.5 * first_box[3])  ,  # ymin = y_center - height/2
#             (first_box[0] + 0.5 * first_box[2])  ,  # xmax = x_center + width/2
#             (first_box[1] + 0.5 * first_box[3])     # ymax = y_center + height/2
#         ], axis=-1)


#         x_center, y_center, width, height = boxes[:4]
#         # 상대 좌표를 절대 픽셀 값으로 변환
#         x_min = (x_center - width / 2) 
#         y_min = (y_center - height / 2) 

#         print(x_min)
#         print(y_min)
#         print(width)
#         print(height)

#         # 이미지 표시
#         fig, ax = plt.subplots(1)
#         ax.imshow(image.numpy())

#         # 첫 번째 바운딩 박스 그리기
#         rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
#         ax.add_patch(rect)

#         plt.show()
#     else:
#         print("Positive 바운딩 박스가 없습니다.")


In [89]:
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# for batch in train_dataset.take(1):
#     images, labels = batch
    
#     # 첫 번째 이미지를 선택
#     image = images[0]
#     image_height, image_width, _ = image.shape

#     # 첫 번째 이미지에 대한 레이블
#     image_labels = labels[0]

#     # Positive 바운딩 박스 추출
#     positive_boxes = image_labels[tf.equal(image_labels[:, 4], 1.0)]

#     # 이미지 표시
#     fig, ax = plt.subplots(1)
#     ax.imshow(image.numpy())

#     # Positive 바운딩 박스를 이미지에 그림
#     for box in positive_boxes:
#         x_center, y_center, width, height = box[:4]
#         # 상대 좌표를 절대 픽셀 값으로 변환
#         x_min = (x_center - width / 2) * image_width
#         y_min = (y_center - height / 2) * image_height
#         width *= image_width
#         height *= image_height

#         # 바운딩 박스 그리기
#         rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
#         ax.add_patch(rect)

#     plt.show()


In [90]:
from keras.layers import Dense, Input, Conv2D, Conv2DTranspose, Flatten, Activation
from keras.layers import Input, GlobalAveragePooling2D, Conv2D, ReLU, Reshape, Multiply
from keras.layers import BatchNormalization, Dropout, ZeroPadding2D
from keras.models import Model
from keras.layers import ZeroPadding2D, DepthwiseConv2D
from keras.regularizers import l2
from keras.layers import Add

class BackBone(tf.keras.Model):
    def __init__(self):
        super(BackBone, self).__init__()
        # self.l2_regularizer = l2(0.001)

    def squeeze_excite_block(self, input_tensor, ratio=8):
        """ Squeeze and Excitation block """
        # 채널 수 얻기
        channels = input_tensor.shape[-1]

        # Squeeze 단계: Global Average Pooling
        x = GlobalAveragePooling2D()(input_tensor)
        x = Reshape((1, 1, channels))(x)

        # Excite 단계: 첫 번째 Conv2D 레이어 (채널 수 축소)
        x = Conv2D(channels // ratio, (1, 1), padding='same')(x)
        x = ReLU()(x)

        # 두 번째 Conv2D 레이어 (채널 수 원래대로 확장)
        x = Conv2D(channels, (1, 1), padding='same')(x)
        # Sigmoid 활성화 함수 적용
        x = tf.keras.activations.sigmoid(x)
        # 원본 입력과 곱하기
        out = Multiply()([input_tensor, x])
        return out

    def residual_layer(self, feature_map, latent, name:str):
        add_layer = Add(name = name+'_output')([feature_map, latent])
        return add_layer

    def feature_extraction_block(self, feature_map, filters_conv1:int, filters_conv2:int, name:str):
        feature_map = Conv2D(filters=filters_conv1, kernel_size = 3, strides = 1, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_1')(feature_map)
        feature_map = Conv2D(filters=filters_conv1, kernel_size = 3, strides = 1, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_2')(feature_map)
        feature_map = BatchNormalization()(feature_map)
        feature_map = Activation('relu')(feature_map)
        feature_map = DepthwiseConv2D(kernel_size=3, padding = 'same')(feature_map)
        feature_map = BatchNormalization()(feature_map)
        # upsample_layer = Conv2DTranspose(filters = 6, kernel_size = 3, strides = (3, 3), padding = 'same')(inputs_image)
        # feature_map = Dropout(0.3)(feature_map)

        feature_map = Conv2D(filters=filters_conv2, kernel_size = 3, strides = 1, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_3')(feature_map)
        feature_map = Conv2D(filters=filters_conv2, kernel_size = 3, strides = 2, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_4')(feature_map)
        feature_map = BatchNormalization()(feature_map)
        feature_map = Activation('relu')(feature_map)
        feature_map = self.squeeze_excite_block(feature_map)
        return feature_map

    def convolutional_residual_block(self, feature_map, filters_conv1:int, filters_conv2:int, name:str):
        latent = Conv2D(filters=filters_conv1, kernel_size = 3, strides = 1, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_1')(feature_map)
        latent = Conv2D(filters=filters_conv1, kernel_size = 3, strides = 1, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_2')(feature_map)
        latent = BatchNormalization()(latent)
        latent = Activation('relu')(latent)
        latent = DepthwiseConv2D(kernel_size=3, padding = 'same')(latent)
        # feature_map = Dropout(0.3)(feature_map)

        latent = Conv2D(filters=filters_conv2, kernel_size = 3, strides = 1, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_3')(latent)
        latent = Conv2D(filters=filters_conv2, kernel_size = 3, strides = 1, padding = 'same', 
                        # kernel_regularizer=self.l2_regularizer,
                        name = name+'_4')(latent)
        latent = BatchNormalization()(latent)
        latent = Activation('relu')(latent)
        latent = DepthwiseConv2D(kernel_size=3, padding = 'same')(latent)
        residual_block = self.residual_layer(feature_map, latent, name)
        return residual_block
    
    def __call__(self, input_shape=(96, 128, 1)):
        inputs_image = Input(shape=input_shape)
        # inputs_image = ZeroPadding2D(((2, 2),(0, 0)))(inputs_image)
        block_1 = self.feature_extraction_block(inputs_image, 16, 32, 'block_1')
        block_1_output = self.convolutional_residual_block(block_1, 16, 32,'block_2')
        block_2 = self.feature_extraction_block(block_1_output, 16, 32, 'block_3')
        block_2_output = self.convolutional_residual_block(block_2, 16, 32, 'block_4')
        block_3 = self.feature_extraction_block(block_2_output, 16, 32,'block_5')
        block_3_output = self.convolutional_residual_block(block_3, 16, 32,'block_6')
        block_4 = self.feature_extraction_block(block_3_output, 16, 32,'block_7')
        block_4_output = self.convolutional_residual_block(block_4, 16, 32,'block_8')

        model = Model(inputs_image, block_4_output)
        return model

In [91]:
backbone = BackBone()
model = backbone()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 96, 128, 1)]         0         []                            
                                                                                                  
 block_1_1 (Conv2D)          (None, 96, 128, 16)          160       ['input_5[0][0]']             
                                                                                                  
 block_1_2 (Conv2D)          (None, 96, 128, 16)          2320      ['block_1_1[0][0]']           
                                                                                                  
 batch_normalization_40 (Ba  (None, 96, 128, 16)          64        ['block_1_2[0][0]']           
 tchNormalization)                                                                          

In [92]:
import keras
def get_backbone():
    backbone = BackBone() 
    backbone = backbone(input_shape=[None, None, 1])

    b2_output, b4_output, b6_output, b8_output= [
        backbone.get_layer(layer_name).output
        for layer_name in ['block_2_output', 'block_4_output', 'block_6_output', 'block_8_output']
    ]

    return keras.Model(
        inputs = [backbone.inputs], outputs=[b2_output, b4_output, b6_output, b8_output]
    )

In [93]:
# block_2_output (Add)        (None, 48, 64, 64)

# block_4_output (Add)        (None, 24, 32, 64)

# block_6_output (Add)        (None, 12, 16, 64)

class FeaturePyramid(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(FeaturePyramid, self).__init__(name="FeaturePyramid", **kwargs)
        self.backbone = get_backbone()
        self.conv_c2_1x1 = keras.layers.Conv2D(32, 1, 1, "same")
        self.conv_c4_1x1 = keras.layers.Conv2D(32, 1, 1, "same")
        self.conv_c6_1x1 = keras.layers.Conv2D(32, 1, 1, "same")
        self.conv_c8_1x1 = keras.layers.Conv2D(32, 1, 1, "same")

        self.conv_c2_3x3 = keras.layers.Conv2D(32, 3, 1, "same")
        self.conv_c4_3x3 = keras.layers.Conv2D(32, 3, 1, "same")
        self.conv_c6_3x3 = keras.layers.Conv2D(32, 3, 1, "same")
        self.conv_c8_3x3 = keras.layers.Conv2D(32, 3, 1, "same")
        
        self.upsample_2x = keras.layers.UpSampling2D(2)
    def call(self, images, training=True):
        b2_output, b4_output, b6_output, b8_output = self.backbone(images, training=training)
        # p2_output = self.conv_c2_1x1(b2_output)
        p4_output = self.conv_c4_1x1(b4_output)
        p6_output = self.conv_c6_1x1(b6_output)
        p8_output = self.conv_c8_1x1(b8_output)
        
        p6_output = p6_output + self.upsample_2x(p8_output)
        p4_output = p4_output + self.upsample_2x(p6_output)
        # p2_output = p2_output + self.upsample_2x(p4_output)
        
        # p2_output = self.conv_c2_3x3(tf.nn.relu(p2_output))
        p4_output = self.conv_c4_3x3(tf.nn.relu(p4_output))
        p6_output = self.conv_c6_3x3(tf.nn.relu(p6_output))
        p8_output = self.conv_c8_3x3(tf.nn.relu(b8_output))
        # print("b2_output shape:", tf.shape(b2_output))
        # print("b4_output shape:", tf.shape(b4_output))
        # print("b6_output shape:", tf.shape(b6_output))
        # print("p2_output shape:", tf.shape(p2_output))
        # print("p4_output shape:", tf.shape(p4_output))
        # print("p6_output shape:", tf.shape(p6_output))
        # print("pn_output shape:", tf.shape(pn_output))

             
        return p4_output, p6_output, p8_output

In [94]:
def build_head(output_filters, bias_init):
    head = keras.Sequential([keras.Input(shape=[None, None, 32])])
    kernel_init = tf.initializers.RandomNormal(0.0, 0.01, seed=1)

    for _ in range(3):
        head.add(
            keras.layers.Conv2D(128, 3, padding = 'same', kernel_initializer=kernel_init)
        )
        head.add(keras.layers.ReLU())

    
    head.add(
        keras.layers.Conv2D(
            output_filters,
            3,
            1,
            padding="same",
            kernel_initializer=kernel_init,
            bias_initializer=bias_init,
        )
    )
    return head

In [95]:
class CustomNet(keras.Model):
    def __init__(self, num_classes, **kwargs):
        super(CustomNet, self).__init__(name="CustomNet", **kwargs)
        self.fpn = FeaturePyramid()
        self.num_classes = num_classes

        prior_probability = tf.constant_initializer(-np.log((1 - 0.01) / 0.01))
        self.cls_head = build_head(8 * num_classes, prior_probability)
        self.box_head = build_head(8 * 4, "zeros")

    def call(self, image, training=True):
        # # # # # # print(f"shape: {image.shape}")
        features = self.fpn(image, training=training)
        N = tf.shape(image)[0]
        cls_outputs = []
        box_outputs = []

        for feature in features:
            box_output = tf.reshape(self.box_head(feature), [N, -1, 4])
            cls_output = tf.reshape(self.cls_head(feature), [N, -1, self.num_classes])

            # print("feature shape:", tf.shape(feature))
            # print("box_output shape:", tf.shape(box_output))
            # print("cls_output shape:", tf.shape(cls_output))

            box_outputs.append(box_output)
            cls_outputs.append(cls_output)
        
        cls_outputs = tf.concat(cls_outputs, axis=1)
        box_outputs = tf.concat(box_outputs, axis=1)
        # print("Final cls_outputs shape:", tf.shape(cls_outputs))
        # print("Final box_outputs shape:", tf.shape(box_outputs))
        final_output = tf.concat([box_outputs, cls_outputs], axis=-1)
        # print("Final output shape:", tf.shape(final_output))
        return final_output

In [96]:
class CustomNetBoxLoss(tf.losses.Loss):  
    def __init__(self, delta):
        super(CustomNetBoxLoss, self).__init__(
            reduction="none", name="CustomNetBoxLoss"
        )
        self._delta = delta

    def call(self, y_true, y_pred):
        difference = y_true - y_pred
        absolute_difference = tf.abs(difference)
        squared_difference = difference ** 2
        loss = tf.where(
            tf.less(absolute_difference, self._delta),
            0.5 * squared_difference,
            absolute_difference - 0.5
        )

        return tf.reduce_sum(loss, axis=-1)

In [97]:
class CustomNetClassificationLoss(tf.losses.Loss):   
    def __init__(self, alpha, gamma):
        super(CustomNetClassificationLoss, self).__init__(
            reduction="none", name="CustomNetClassificationLoss"
        )
        self._alpha = alpha
        self._gamma = gamma

    def call(self, y_true, y_pred):
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=y_true, logits=y_pred
        )
        probs = tf.nn.sigmoid(y_pred)
        alpha = tf.where(tf.equal(y_true, 1.0), self._alpha, (1.0 - self._alpha))
        pt = tf.where(tf.equal(y_true, 1.0), probs, 1 - probs)
        loss = alpha * tf.pow(1.0 - pt, self._gamma) * cross_entropy

        return tf.reduce_sum(loss, axis=-1)

In [98]:
class CustomNetLoss(tf.losses.Loss):    
    def __init__(self, num_classes=1, alpha=0.25, gamma=2.0, delta=1.0):
        super(CustomNetLoss, self).__init__(reduction="auto", name="CustomNetLoss")
        self._cls_loss = CustomNetClassificationLoss(alpha, gamma)
        self._box_loss = CustomNetBoxLoss(delta)
        self._num_classes = num_classes

    def call(self, y_true, y_pred):
        # print("y_true shape:", tf.shape(y_true))
        # print("y_pred shape:", tf.shape(y_pred))

        y_pred = tf.cast(y_pred, dtype=tf.float32)
        box_labels = y_true[:, :, :4]
        box_predictions = y_pred[:, :, :4]
        # print("box_labels shape:", tf.shape(box_labels))
        # print("box_predictions shape:", tf.shape(box_predictions))
        
        cls_labels = tf.one_hot(
            tf.cast(y_true[:, :, 4], dtype=tf.int32),
            depth=self._num_classes,
            dtype=tf.float32,
        )
        cls_predictions = y_pred[:, :, 4:]
        # print("cls_labels shape:", tf.shape(cls_labels))
        # print("cls_predictions shape:", tf.shape(cls_predictions))

        positive_mask = tf.cast(tf.greater(y_true[:, :, 4], -1.0), dtype=tf.float32)
        ignore_mask = tf.cast(tf.equal(y_true[:, :, 4], -2.0), dtype=tf.float32)
        # print("positive_mask shape:", tf.shape(positive_mask))
        # print("ignore_mask shape:", tf.shape(ignore_mask))

        cls_loss = self._cls_loss(cls_labels, cls_predictions)
        box_loss = self._box_loss(box_labels, box_predictions)
        # print("cls_loss shape:", tf.shape(cls_loss))
        # print("box_loss shape:", tf.shape(box_loss))

        cls_loss = tf.where(tf.equal(ignore_mask, 1.0), 0.0, cls_loss)
        box_loss = tf.where(tf.equal(positive_mask, 1.0), box_loss, 0.0)
        
        normalizer = tf.reduce_sum(positive_mask, axis=-1)
        # print("normalizer:", normalizer)

        cls_loss = tf.math.divide_no_nan(tf.reduce_sum(cls_loss, axis=-1), normalizer)
        box_loss = tf.math.divide_no_nan(tf.reduce_sum(box_loss, axis=-1), normalizer)
        
        loss = cls_loss + box_loss
        # print("Final loss shape:", tf.shape(loss))
        
        return loss

In [99]:
# import tensorflow as tf
# from tensorflow.keras.losses import MeanSquaredError, CategoricalCrossentropy

# class CustomNetLoss(tf.losses.Loss):
#     def __init__(self, num_classes=1):
#         super(CustomNetLoss, self).__init__(reduction="auto", name="CustomNetLoss")
#         self._num_classes = num_classes
#         self.mse_loss = MeanSquaredError(reduction="none")
#         self.cce_loss = CategoricalCrossentropy(reduction="none", from_logits=True)

#     def call(self, y_true, y_pred):
#         # 박스 레이블과 예측 분리
#         box_labels = y_true[:, :, :4]
#         box_predictions = y_pred[:, :, :4]

#         # 클래스 레이블과 예측 분리 및 one-hot 인코딩
#         cls_labels = tf.one_hot(tf.cast(y_true[:, :, 4], dtype=tf.int32), depth=self._num_classes)
#         cls_predictions = y_pred[:, :, 4:]

#         # 박스 손실(MSE) 계산
#         box_loss = self.mse_loss(box_labels, box_predictions)

#         # 클래스 손실(Categorical Cross-Entropy) 계산
#         cls_loss = self.cce_loss(cls_labels, cls_predictions)

#         # 손실 합산
#         total_loss = tf.reduce_mean(cls_loss) + tf.reduce_mean(box_loss)
#         return total_loss


In [100]:
num_classes = 1

learning_rates = [2.5e-06, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values=learning_rates
)

# LR = 0.0005
# initial_learning_rate = LR
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=5000,
#     decay_rate=0.90,
#     staircase=True)


model_dir = "ObjectDetectionCheckpoint/customnet.ckpt"

In [101]:
from keras.metrics import Precision, Recall

# # # # 멀티 GPU 전략 설정
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # 모델, 손실 함수, 옵티마이저를 전략 범위 내에서 정의
    model = CustomNet(num_classes)
    loss_fn = CustomNetLoss()
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)

    # 모델 컴파일
    model.compile(optimizer=optimizer, 
                  loss=loss_fn,
                  metrics=['accuracy', Precision(), Recall()])


# # # # # 모델, 손실 함수, 옵티마이저를 전략 범위 내에서 정의
# model = CustomNet(num_classes)
# loss_fn = CustomNetLoss()
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
# # 모델 컴파일
# model.compile(optimizer=optimizer, 
#               loss=loss_fn,
#               metrics=['accuracy', Precision(), Recall()])


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/

In [102]:
callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
        monitor="loss",
        save_best_only=True,
        save_weights_only=True,
        verbose=1,
    )
]

In [103]:
# train_dataset에서 하나의 배치를 가져옵니다
for batch in train_dataset.take(1):
    # 배치에서 이미지와 레이블을 추출합니다
    images, labels = batch
    
    # 이미지와 레이블의 형태를 출력합니다
    print("Images shape:", images.shape)
    print("Images max:", tf.reduce_max(images))
    print("Images min:", tf.reduce_min(images))
    print("Labels shape:", labels.shape)

Images shape: (7, 96, 128, 1)
Images max: tf.Tensor(1.0, shape=(), dtype=float32)
Images min: tf.Tensor(0.0, shape=(), dtype=float32)
Labels shape: (7, 8064, 5)


In [104]:
# q!

In [105]:
# import tensorflow as tf

# # TensorFlow Dataset 예시
# # train_dataset = tf.data.Dataset.from_tensor_slices(...)

# def check_nan_in_dataset(dataset):
#     for batch in dataset:
#         images, labels = batch
#         if tf.reduce_any(tf.math.is_nan(images)) or tf.reduce_any(tf.math.is_nan(labels)):
#             return True
#     # print(images)
#     # print(labels)
#     return False


# contains_nan = check_nan_in_dataset(train_dataset)
# if contains_nan:
#     print("Data contains NaN values")
# else:
#     print("Data does not contain NaN values")

# check_nan_in_dataset(train_dataset)
# check_nan_in_dataset(val_dataset)


In [106]:
epochs = 100

model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks_list,
    verbose=1,
)

2024-01-23 14:16:47.786171: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "UnbatchDataset/_28"
op: "UnbatchDataset"
input: "MapDataset/_27"
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021UnbatchDataset:69"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 96
        }
        dim {
          size: 128
        }
        dim {
          size: 1
        }
      }
      shape {
        dim {
          size: 8200
        }
        dim {
          size: 5
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
  

Epoch 1/100
INFO:tensorflow:Collective all_reduce tensors: 164 all_reduces, num_devices = 7, group_size = 7, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 164 all_reduces, num_devices = 7, group_size = 7, implementation = CommunicationImplementation.NCCL, num_packs = 1


2024-01-23 14:18:08.415236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905
2024-01-23 14:18:09.594885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905
2024-01-23 14:18:10.838538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905
2024-01-23 14:18:12.186020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905
2024-01-23 14:18:13.848318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905
2024-01-23 14:18:14.728375: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905


    311/Unknown - 127s 77ms/step - loss: 1.2941 - accuracy: 0.1758 - precision_1: 0.1489 - recall_1: 9.9417e-04
Epoch 1: loss improved from inf to 1.29408, saving model to ObjectDetectionCheckpoint/customnet.ckpt/weights_epoch_1


2024-01-23 14:18:56.372610: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3417217166738408402
2024-01-23 14:18:56.372695: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 15197218983853130882
2024-01-23 14:18:56.372731: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 84200149573206411
2024-01-23 14:18:56.372799: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 14572776815660160788
2024-01-23 14:18:56.372868: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 18193135748813391130
2024-01-23 14:18:56.372917: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 16176776346855867109
2024-01-23 14:18:56.372984: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv i

311/311 [==============================] - 128s 80ms/step - loss: 1.2941 - accuracy: 0.1758 - precision_1: 0.1489 - recall_1: 9.9417e-04
Epoch 2/100
311/311 [==============================] - ETA: 0s - loss: 1.0550 - accuracy: 0.1905 - precision_1: 0.1442 - recall_1: 0.0031
Epoch 2: loss improved from 1.29408 to 1.05498, saving model to ObjectDetectionCheckpoint/customnet.ckpt/weights_epoch_2
311/311 [==============================] - 25s 79ms/step - loss: 1.0550 - accuracy: 0.1905 - precision_1: 0.1442 - recall_1: 0.0031
Epoch 3/100
311/311 [==============================] - ETA: 0s - loss: 0.9082 - accuracy: 0.4075 - precision_1: 0.1390 - recall_1: 0.0057
Epoch 3: loss improved from 1.05498 to 0.90815, saving model to ObjectDetectionCheckpoint/customnet.ckpt/weights_epoch_3
311/311 [==============================] - 25s 80ms/step - loss: 0.9082 - accuracy: 0.4075 - precision_1: 0.1390 - recall_1: 0.0057
Epoch 4/100
311/311 [==============================] - ETA: 0s - loss: 0.8846 - a

In [ ]:
model.summary()

In [ ]:
q!

In [ ]:
# class DecodePredictions(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         num_classes=1,
#         confidence_threshold=0.05,
#         nms_iou_threshold=0.5,
#         max_detections_per_class=1,
#         max_detections=4,
#         box_variance=[0.1, 0.1, 0.2, 0.2],
#         **kwargs
#     ):
#         super(DecodePredictions, self).__init__(**kwargs)
#         self.num_classes = num_classes
#         self.confidence_threshold = confidence_threshold
#         self.nms_iou_threshold = nms_iou_threshold
#         self.max_detections_per_class = max_detections_per_class
#         self.max_detections = max_detections

#         self._anchor_box = AnchorBox()
#         self._box_variance = tf.convert_to_tensor(
#             [0.1, 0.1, 0.2, 0.2], dtype=tf.float32
#         )

#     def _decode_box_predictions(self, anchor_boxes, box_predictions):
#         boxes = box_predictions * self._box_variance
#         boxes = tf.concat(
#             [
#                 boxes[:,:,:2] * anchor_boxes[:,:,2:] + anchor_boxes[:,:,:2],
#                 tf.math.exp(boxes[:,:,2:]) * anchor_boxes[:,:,2:]
#             ],
#             axis=-1,
#         )
#         boxes_transformed = convert_to_corners(boxes)
#         return boxes_transformed

#     def call(self, images, predictions):
#         image_shape = tf.cast(tf.shape(images), dtype=tf.float32)
#         anchor_boxes = self._anchor_box.get_anchors(image_shape[1], image_shape[2])
#         box_predictions = predictions[:,:,:4]
#         cls_predictions = predictions[:,:,4:]
#         boxes = self._decode_box_predictions(anchor_boxes[None, ...], box_predictions)
#         cls_predictions = tf.nn.sigmoid(cls_predictions)

#         return tf.image.combined_non_max_suppression(
#             tf.expand_dims(boxes, axis=2),
#             cls_predictions,
#             self.max_detections_per_class,
#             self.max_detections,
#             self.nms_iou_threshold,
#             self.confidence_threshold,
#             clip_boxes=False,
#         )

In [ ]:
# image = tf.keras.Input(shape=[None, None, 1], name="image")
# predictions = model(image, training=False)
# # detections = DecodePredictions(confidence_threshold=0.5)(image, predictions)
# inference_model = tf.keras.Model(inputs=image, outputs=predictions)

In [ ]:
# def visualize_detections(
#     image, boxes, classes, scores, figsize=(7, 7), linewidth=1, color=[0, 0, 1]
# ):
#     image = np.array(image, dtype=np.uint8)
#     plt.figure(figsize=figsize)
#     plt.axis("off")
#     plt.imshow(image)
#     ax = plt.gca()
#     for box, _cls, score in zip(boxes, classes, scores):
#         text = "{}: {:.2f}".format(_cls, score)
#         x1, y1, x2, y2 = box
#         w, h = x2 - x1, y2 - y1
#         patch = plt.Rectangle(
#             [x1, y1], w, h, fill=False, edgecolor=color, linewidth=linewidth
#         )
#         ax.add_patch(patch)
#         ax.text(
#             x1,
#             y1,
#             text,
#             bbox={"facecolor": color, "alpha": 0.4},
#             clip_box=ax.clipbox,
#             clip_on=True,
#         )
#     plt.show()
#     return ax

In [ ]:
# image = []
# for img, _ in train_dataset.take(1):
#     image.append(img[0])
# image = np.array(image)
# plt.imshow(image[0])
# plt.show()

In [ ]:
a = dataset['images']
plt.imshow(a[3000])
plt.show()


In [ ]:
# def prepare_image(image):
#     image, _, ratio = resize_and_pad_image(image)
#     # image = tf.keras.applications.resnet.preprocess_input(image)
#     return tf.expand_dims(image, axis=0), ratio


# image = dataset['images']
# image = tf.cast(image[3000], dtype=tf.float32)
# input_image, ratio = prepare_image(image)
# print(input_image.shape)
# print(ratio)
# detections = inference_model.predict(input_image)
# print(detections.shape)
# # num_detections = detections.valid_detections[0]
# # visualize_detections(
#     # image,
#     # detections.nmsed_boxes[0][:num_detections] / ratio,
#     # detections.nmsed_scores[0][:num_detections],
# # )